# Добавление необходимой библиотеки

In [55]:
import numpy as np

# Создание матриц для игры

In [56]:
table = np.array([
    [
        [3, 3, 2],
        [7, 2, 4],
        [1, 5, 3]
    ],
    [
        [4, 1, 3],
        [3, 4, 5],
        [2, 6, 4]
    ],
])
table

array([[[3, 3, 2],
        [7, 2, 4],
        [1, 5, 3]],

       [[4, 1, 3],
        [3, 4, 5],
        [2, 6, 4]]])

# Равновесие в доминирующих стратегиях

In [57]:
def find_dominate_line(matrix):  # Функция для нахождения доминирующей строки
    dominated = set()
    been_dominating = set()

    for i in range(len(matrix)):
        for j in range(len(matrix)):
            if i == j:
                continue
            if all(map(lambda x: x[0] < x[1], zip(matrix[i], matrix[j]))):
                dominated.add(j)
                been_dominating.add(i)
    result = list(dominated.difference(been_dominating))
    return None if len(result) == 0 else result[0]

In [58]:
dominants = [find_dominate_line(table[i]) for i in range(len(table))]
print(dominants)

if len(set(dominants)) == len([i for i in dominants if not i is None]):
    print('Равновесие соблюдено')
else:
    print('Равновесие не соблюдено')

[None, None]
Равновесие не соблюдено


# Равновесие по Нэшу

In [62]:
A = np.argmax(table[0], axis=0)
B = np.argmax(table[1], axis=1)
nesh = []
for i in range(len(A)):
    if A[i] == B[i]:
        nesh.append((A[i], B[i]))

print(f'Точки равновесия по Нэшу: {nesh}')

Точки равновесия по Нэшу: [(2, 2), (1, 1)]


In [64]:
converted_table = [[(None,) for j in range(len(table[0][i]))] for i in range(len(table[0]))]
for i in range(len(converted_table)):
    for j in range(len(converted_table[i])):
        converted_table[i][j] = tuple([table[k][i][j] for k in range(len(table))])
converted_table

[[(3, 4), (3, 1), (2, 3)], [(7, 3), (2, 4), (4, 5)], [(1, 2), (5, 6), (3, 4)]]

# Равновесие по Парето

In [65]:
print(f'Точки равновесия по Парето: {[i for i in nesh if converted_table[i[1]][i[0]][0] + converted_table[i[1]][i[0]][1] == max([converted_table[j[1]][j[0]][0] + converted_table[j[1]][j[0]][1] for j in nesh])]}')

Точки равновесия по Парето: [(2, 2)]


# Равновесие по Штакельбергу

In [66]:
target_values = [converted_table[p[1]][p[0]] for p in nesh]
target_value

4